In [ ]:
# 运算符： 中缀运算符、一元运算符，函数调用()、属性访问.和元素访问/切片[]也是运算符

# 运算符重载：
# - 不能重载内置类型的运算符
# - 不能新建运算符，只能重载已有的
# - is、and、or和not不能重载


from array import array
import reprlib
import math
import numbers
import functools
import operator
import itertools


class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __eq__(self, other): # 重载运算符 ==
        if isinstance(other, Vector):  # 类型判断 
            return (len(self) == len(other) and
                    all(a == b for a, b in zip(self, other)))
        else:
            return NotImplemented  # 返回此结果，解释器会尝试用调用other的__eq__方法

    def __hash__(self):
        hashes = (hash(x) for x in self)
        return functools.reduce(operator.xor, hashes, 0)

    def __abs__(self): # abs也是运算符
        return math.sqrt(sum(x * x for x in self))

    def __neg__(self): # 重载运算符 ~
        return Vector(-x for x in self) # 构建新的副本

    def __pos__(self): # 重载运算符 +
        return Vector(self) # 构建新的副本

    def __bool__(self):
        return bool(abs(self))

    def __len__(self):
        return len(self._components)

    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{.__name__} indices must be integers'
            raise TypeError(msg.format(cls))

    shortcut_names = 'xyzt'

    def __getattr__(self, name):
        cls = type(self)
        if len(name) == 1:
            pos = cls.shortcut_names.find(name)
            if 0 <= pos < len(self._components):
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))

    def angle(self, n):
        r = math.sqrt(sum(x * x for x in self[n:]))
        a = math.atan2(r, self[n-1])
        if (n == len(self) - 1) and (self[-1] < 0):
            return math.pi * 2 - a
        else:
            return a

    def angles(self):
        return (self.angle(n) for n in range(1, len(self)))

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('h'):  # hyperspherical coordinates
            fmt_spec = fmt_spec[:-1]
            coords = itertools.chain([abs(self)],
                                     self.angles())
            outer_fmt = '<{}>'
        else:
            coords = self
            outer_fmt = '({})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(', '.join(components))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)
    
    # a+b，如果a有__add__方法，且返回值不是NotImplemented，则调用；否则检查b是否有__radd__方法，如果有且没有返回NotImplemented，
    # 则调用b.__radd__(a)；否则抛出TypeError异常。
    
    # 实现 __add__方法后，缺省就实现了 a += b为 a = a+b，如果要重载运算符 +=，则需要实现 __iadd__方法
    def __add__(self, other): # 重载中缀运算符 +，
        try:
            pairs = itertools.zip_longest(self, other, fillvalue=0.0) # 元组生成器，调用zip_longest 会给短的补齐指定数值
            return Vector(a + b for a, b in pairs) # 构建新Vector对象
        except TypeError:
            return NotImplemented

    def __radd__(self, other): # 后备方法，是__add__的反射 refelcted/reversed版本，或右向特殊方法 right method
        return self + other

    # a*b 调用机制与a+b类似
    def __mul__(self, scalar): # 重载标量乘法运算符 *
        if isinstance(scalar, numbers.Real): # 检查是否是标量
            return Vector(n * scalar for n in self)
        else:
            return NotImplemented

    def __rmul__(self, scalar): # 后备方法
        return self * scalar

    # Python 3.5新引入的运算符@ 向量点积运算符
    def __matmul__(self, other): # 重载点积运算符 @
        try:
            return sum(a * b for a, b in zip(self, other))
        except TypeError:
            return NotImplemented

    def __rmatmul__(self, other): # 后备方法
        return self @ other  # this only works in Python 3.5

v1 = Vector([3, 4, 5])
v2 = Vector([6, 7, 8])
v3 = Vector([1, 2])
print(format(v1+v2, '0.2f'))
print(format(v1+v3, '0.2f'))
print(format(v1+(10, 20, 30), '0.2f'))
print(format(v1*2.5, '0.2f'))
print((v1+v2)==v3)
print(v1==[3, 4, 5])
print(v1==(3, 4, 5))
print((v1+v2)!=v3)
print(format(v1@v3, '0.2f'))
    
